In [9]:
import pandas as pd
import json
import altair as alt
import requests

Untersucht die Stimmanteile der Grünen Liste im Kanton Bern in den Nationalratswahlen 2019

In [465]:
# downloaded from https://www.bfs.admin.ch/bfsstatic/dam/assets/10307883/master
kandidaten_df = pd.json_normalize(json.load(open('sd-t-17.02-NRW2019-kandidierende-gemeinden.json', encoding='utf-8'))['SASTableData+NRW2019_KANDIDIERENDE_GEMEINDEN'])
kandidaten_gruene_bern_df = kandidaten_df[(kandidaten_df['KANTON_ID'] == 2) & (kandidaten_df['LISTE_NUMMER'] == 12)]
kandidaten_filtered_df = kandidaten_gruene_bern_df[(kandidaten_df['KANDIDAT_NUMMER'].isin((13,14,24)))]

C:\Users\U808595\AppData\Local\Temp\ipykernel_5816\1159790132.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  kandidaten_filtered_df = kandidaten_gruene_bern_df[(kandidaten_df['KANDIDAT_NUMMER'].isin((13,14,24)))]


In [166]:
kandidaten_sum_pivot_df = kandidaten_filtered_df.pivot_table(index='GEMEINDE_NUMMER', columns=['NAME'], values='STIMMEN_KANDIDAT', aggfunc='sum')
# kandidaten_sum_pivot_df[kandidaten_sum_pivot_df['KANTON_ID']]
# kandidaten_sum_pivot_df[kandidaten_sum_pivot_df['Gutzwiller']]
# kandidaten_sum_pivot_df[['NAME']]
kandidaten_sum_pivot_df = kandidaten_sum_pivot_df.reset_index(drop=False)

In [13]:
topojson = requests.get('https://dam-api.bfs.admin.ch/hub/api/dam/assets/22344268/master', verify=False).json()

C:\dev\tools\scoop\apps\python\current\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dam-api.bfs.admin.ch'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [38]:
# nur gemeinden im kanton bern behalten
topojson['objects']['K4voge_20220501_gf']['geometries'] = [g for g in topojson['objects']['K4voge_20220501_gf']['geometries'] if g['properties']['kantName'] == 'Bern']

In [565]:
stimmen_pro_gemeinde_df = kandidaten_df[(kandidaten_df['KANTON_ID'] == 2) & (kandidaten_df['LISTE_NUMMER'] == 12)][['GEMEINDE_NUMMER', 'STIMMEN_KANDIDAT']].groupby(by='GEMEINDE_NUMMER', as_index=False).median()

In [566]:
from altair import datum

variable_list = ['Gutzwiller', 'Hilty Haller', 'Widmer']
temp_df = kandidaten_gruene_bern_df[(kandidaten_gruene_bern_df['NAME'].isin(variable_list))].reset_index(drop=True).merge(stimmen_pro_gemeinde_df, left_on='GEMEINDE_NUMMER', right_on='GEMEINDE_NUMMER')
temp_df['STIMMEN_RELATIV'] = (temp_df['STIMMEN_KANDIDAT_x'] - temp_df['STIMMEN_KANDIDAT_y'])

data = alt.InlineData(values=topojson, format=alt.DataFormat(feature='K4voge_20220501_gf', type='topojson'))
# selector = alt.selection_single(empty='all', fields=['properties.vogeId'])

base = alt.Chart(data).mark_geoshape(stroke='black',strokeWidth=.5,fill=None).encode(
).project(
    type='identity', reflectY=True
).properties(
    width=800,
    height=500
)

gemeinden = alt.Chart(temp_df).mark_geoshape(stroke='black',strokeWidth=.5).encode(
    color=alt.Color(
            'STIMMEN_RELATIV:Q',
            scale=alt.Scale(scheme='blueorange', domain=[-100,100], clamp=True),
            legend=alt.Legend(title='Stimmen relativ Median')
    ),
    tooltip=[
        alt.Tooltip('properties.vogeName:N', title='Gemeinde'),
        alt.Tooltip('STIMMEN_KANDIDAT_x:Q', title='Stimmen Kandidat'),
        alt.Tooltip('STIMMEN_KANDIDAT_y:Q', title='Stimmen Median', format='.1f'),
        alt.Tooltip('STIMMEN_RELATIV:Q', title='Differenz', format='.1f'),
        # alt.Tooltip(alt.repeat('row'), type='quantitative', format='.2%', title='Stimmanteil Liste 12'),
    ],
# ).transform_filter(
#     datum['NAME'] == 'Widmer'
).transform_lookup(
    lookup='GEMEINDE_NUMMER',
    from_=alt.LookupData(data, 'properties.vogeId', ["type", "properties", "geometry"])
).project(
    type='identity', reflectY=True
).properties(
    # title=alt.TitleParams(alt.repeat('row')),
    width=800,
    height=500,
)

chart = alt.vconcat()
for name in variable_list:
    chart &= gemeinden.transform_filter(datum['NAME'] == name).properties(title=name)
chart

alt.VConcatChart(...)

In [563]:
df = kandidaten_df[(kandidaten_df['KANTON_ID'] == 2)].groupby(by='GEMEINDE_NUMMER', as_index=False).sum()[['GEMEINDE_NUMMER', 'STIMMEN_KANDIDAT']].reset_index(drop=True)
overview_df = kandidaten_df[(kandidaten_df['KANTON_ID'] == 2)][['GEMEINDE_NUMMER','LISTE_NUMMER','STIMMEN_KANDIDAT']].groupby(by=['GEMEINDE_NUMMER','LISTE_NUMMER'], as_index=False).sum().reset_index(drop=True)
overview_df = overview_df.merge(df, on='GEMEINDE_NUMMER')
overview_df['LISTE_STIMMEN_ANTEIL'] = overview_df['STIMMEN_KANDIDAT_x'] / overview_df['STIMMEN_KANDIDAT_y']

chart = alt.Chart(overview_df[(overview_df['LISTE_NUMMER'] == 12)]).mark_geoshape(stroke='black',strokeWidth=.5).encode(
    color=alt.Color(
        'LISTE_STIMMEN_ANTEIL:Q',
        scale=alt.Scale(scheme='blueorange'),
        legend=alt.Legend(title='Anteil in %', format='.1%')
    ),
    tooltip=[
        alt.Tooltip('properties.vogeName:N', title='Gemeinde'),
        alt.Tooltip('LISTE_STIMMEN_ANTEIL:Q', title='Anteil', format='.1%'),
        alt.Tooltip('STIMMEN_KANDIDAT_x:Q', title='Stimmen Liste'),
        alt.Tooltip('STIMMEN_KANDIDAT_y:Q', title='Stimmen Total'),
    ],
).transform_lookup(
    lookup='GEMEINDE_NUMMER',
    from_=alt.LookupData(data, 'properties.vogeId', ["type", "properties", "geometry"])
).project(
    type='identity', reflectY=True
).properties(
    title='Stimmenanteil Grüne - Nationalratswahlen 2019',
    width=800,
    height=500,
)
chart

alt.Chart(...)

Die Grünen sind in Bern und Biel sehr gut vertreten, in ländlicheren Gegenden weniger so.